5.1 全链接层
全链接输入与输出矩阵格式为[BATCHSIZE, N]， 假设网络某一层输入为 x^l ，输出为 x^{l+1} ，那么输入与输出层间关系为：

\begin{matrix} h^l=x^l\cdot W&(a)\\ u^l=h^l+b&(b)\\ x^{l+1}=f(u^l)&(c)\\ f(\cdot)\rightarrow {sigmoid}&(d)\\ sigmoid(x)=\frac{1}{1+e^{-x}}&(e) \end{matrix} (1.1)

5.2 链式求导
链式求导是目前为止整个深度学习的基础，有人将链式求导法则称之为反向传播。反向传播是从loss函数开始的。

传播过程之中每一层均会计算两个内容：
1. 本层可训练参数的导数，
2. 本层向前传播误差（链式求导）。

举个例子来说：

 \begin{matrix} loss=(y-d)^2&(a)\\ y=f_1(a\cdot f_2(b\cdot f_3(c\cdot x)))&(b)\\ error1=\frac{\partial loss}{\partial y}=2 (y - d)&(c)\\ error2=\frac{\partial loss}{\partial (a\cdot f_2)}=error1\cdot f_1'&(d)\\ \frac{\partial loss}{\partial a}=error2\cdot f_2&(e)\\ error3=\frac{\partial loss}{\partial (f_2)}=error2\cdot a&(f)\\ \end{matrix}  (1.2)

对于 a....f_2 这一层来说，有一个可训练参数a，那么反向传播需要计算可训练参数a的导数1.1-(e)，同时为了计算 f_3 之中的可训练参数，此层需要产生新的error3。对于 f_1(\cdot) 这一层来说，由于没有可训练参数，因此仅产生反向传播误差error2。
注意，1.2中将每一步计算，包括相乘、相加、通过函数均算为单独的计算层。因此全链接层包括：矩阵相乘（wx）-矩阵相加（wx+b）-函数计算（f(wx+b)）三个计算层。

#### Use Matrix

In [ ]:
 def _matmul(self, inputs, W, *args, **kw):
        """
        正向传播
        """
        return np.dot(inputs, W)
    def _d_matmul(self, in_error, n_layer, layer_par):
        """
        反向传播
        """
        W = self.value[n_layer]
        inputs = self.outputs[n_layer]
        self.d_value[n_layer] = np.dot(inputs.T, in_error)
        error = np.dot(in_error, W.T)
        return error
    def matmul(self, filters, *args, **kw):
        self.value.append(filters)
        self.d_value.append(np.zeros_like(filters))
        self.layer.append((self._matmul, None, self._d_matmul, None))
        self.layer_name.append("matmul")

#### Add_Bias

In [ ]:
def _bias_add(self, inputs, b, *args, **kw):
        return inputs + b
    def _d_bias_add(self, in_error, n_layer, layer_par):
        self.d_value[n_layer] = np.sum(in_error, axis=0)
        return in_error
    def bias_add(self, bias, *args, **kw):
        self.value.append(bias)
        self.d_value.append(np.zeros_like(bias))
        self.layer.append((self._bias_add, None, self._d_bias_add, None))
        self.layer_name.append("bias_add")

#### Activation Function

In [ ]:
def _sigmoid(self, X, *args, **kw):
        return 1/(1+np.exp(-X))
    def _d_sigmoid(self, in_error, n_layer, *args, **kw):
        X = self.outputs[n_layer]
        return in_error * np.exp(-X)/(1 + np.exp(-X)) ** 2
    def sigmoid(self):
        self.value.append([])
        self.d_value.append([])
        self.layer.append((self._sigmoid, None, self._d_sigmoid, None))
        self.layer_name.append("sigmoid")

#### Loss Function

In [ ]:
def _loss_square(self, Y, *args, **kw):
        B = np.shape(Y)[0]
        return np.square(self.outputs[-1] - Y)/B
    def _d_loss_square(self, Y, *args, **kw):
        B = np.shape(Y)[0]
        return 2 * (self.outputs[-2] - Y)
    def loss_square(self):
        self.value.append([])
        self.d_value.append([])
        self.layer.append((self._loss_square, None, self._d_loss_square, None))    
        self.layer_name.append("loss")

#### Train : 

w_new <- w_old + beta dot  dw

In [ ]:
def forward(self, X):
    self.outputs = []
    self.outputs.append(X)
    net = X
    for idx, lay in enumerate(self.layer):
        method, layer_par, _, _ = lay
        net = method(net, self.value[idx], layer_par)
        self.outputs.append(net)
    return
def backward(self, Y):
    error = self.layer[-1][2](Y)
    self.n_layer = len(self.value)
    for itr in range(self.n_layer-2, -1, -1):
        _, _, method, layer_par = self.layer[itr]
        error = method(error, itr, layer_par)
def apply_gradient(self, eta):
    for idx, itr in enumerate(self.d_value):
        if len(itr) == 0: continue
        self.value[idx] -= itr * eta
def fit(self, X, Y):
    self.forward(X)
    self.backward(Y)
    self.apply_gradient(0.1)
def predict(self, X):
    self.forward(X)
    return self.outputs[-2]

#### Run

In [ ]:
class NN():
    def __init__(self):
        self.value = []
        self.d_value = []
        # 每一层输出
        self.outputs = []
        # 每一层所用函数
        self.layer = []
        # 层名
        self.layer_name = []

In [ ]:
###### 初始化值
iw1 = np.random.normal(0, 0.1, [28, 28])
ib1 = np.zeros([28])
iw2 = np.random.normal(0, 0.1, [28, 28])
ib2 = np.zeros([28])
iw3 = np.random.normal(0, 0.1, [28, 2])
ib3 = np.zeros([2])
##### 神经网络描述
mtd = NN()
mtd.matmul(iw1)
mtd.bias_add(ib1)
mtd.sigmoid()
mtd.matmul(iw2)
mtd.bias_add(ib2)
mtd.sigmoid()
mtd.matmul(iw3)
mtd.bias_add(ib3)
mtd.sigmoid()
mtd.loss_square()
###### 训练
for itr in range(100):
    ...
    mtd.fit(inx, iny)


8 运行结果
输出模型：
Layer 0: matmul
Layer 1: bias_add
Layer 2: sigmoid
Layer 3: matmul
Layer 4: bias_add
Layer 5: sigmoid
Layer 6: matmul
Layer 7: bias_add
Layer 8: sigmoid
Layer 9: loss
迭代100次后精度96%。

Thanks for https://zhuanlan.zhihu.com/p/37025766